In [1]:
!pip install pymorphy2
!pip install summa
!pip install keybert
!pip3 install python-rake

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.8 MB/s 
     |████████████████████████████████| 8.2 MB 14.5 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=a26a1df43b65e8dabadc47383b8d2f898b27a44e50bb3282b8dbf21766a5ee51
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built docopt
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 54 kB 2.5 MB/s 
  Created wheel for summa: filename=summa-1.2.0-py3-none-any.whl size=54410 sha256=0776ae4679f9c02445d1e5644368f04ef7bbe8584924aa399a89b5e41b41e593
  Stored in directory: /root/.cache/pip/wheels/29/64/ac/7b443477588d365ef37ada30d456bdf5f07dc5be9f6324cb6e
Successfully built summa
Looking in indexes: https://pypi.org/simple, https://us-python.pk

In [2]:
import pandas as pd
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import pymorphy2
import string
from string import punctuation, digits
from tqdm.notebook import tqdm
tqdm.pandas()
import RAKE
from summa import keywords
from keybert import KeyBERT
from collections import Counter

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
corpus = pd.read_csv('corpus.csv', sep = ';', names = ['title', 'text', 'tags'])
tags_manual = pd.read_csv('tags_manual.csv', sep = ',') 
corpus_full = pd.merge(corpus, tags_manual, left_on=['title'], right_on=['title'], how='left')

In [4]:
tags = [set(tags.split(', ')) for tags in list(corpus_full['tags'])]
tags_manual = [set(tags.split(', ')) for tags in list(corpus_full['tags_manual'])]
for i in range(0, len(tags)):
    print(list(tags[i] & tags_manual[i]))

[]
[]
[]
['рабочие места']
[]
[]
[]
[]
[]
['программа реновации']
['современное искусство']
[]
[]
[]
['мониторинг', 'автоматизация']
[]
['Московская техническая школа']
['программа реновации']
[]
['показания счетчиков', 'Моя Москва']


In [5]:
with open('merge_tags.txt', 'w') as f:
    for i in range(0, len(tags)):
        f.write(str(tags[i] | tags_manual[i]) + '\n')

#между этими шагами произошло немного ручной работы, а именно выбор оптимальных тегов

with open('gold.txt', 'r') as f:
    tags_gold = f.read().split('\n')

corpus_full['tags_gold'] = tags_gold

In [6]:
def clear_and_morhp(text, stop_words, morph): 
    text = [word.lower().translate(str.maketrans('', '', punctuation)) for word in text.split()]
    text = [word.translate(str.maketrans('', '', digits)) for word in text]
    text = [word for word in text if word != '']
    text = [word for word in text if word not in stop_words]
    
    lemmas = []
    for word in text:
        result = morph.parse(word)[0].normal_form
        lemmas.append(result)

    return ' '.join(lemmas)

morph = pymorphy2.MorphAnalyzer()
stop_words = stopwords.words("russian")
corpus_full['text_clean'] = corpus_full['text'].progress_apply(lambda x: clear_and_morhp(x, stop_words, morph))

  0%|          | 0/20 [00:00<?, ?it/s]

In [7]:
rake = RAKE.Rake(stop_words)
def use_rake(text):
  kw_list = rake.run(text, maxWords=3, minFrequency=1)
  final_kw = []
  for kw in kw_list:
    if kw[-1] >= 2:
      final_kw.append(kw)
  print(final_kw)
  list_kw = []
  for k in final_kw:
    list_kw.append(k[0])

  return ', '.join(list_kw)

corpus_full['rake'] = corpus_full['text'].progress_apply(lambda x: use_rake(x))

  0%|          | 0/20 [00:00<?, ?it/s]

[('будут кратно выше', 9.0), ('сергей собянин сказал', 9.0), ('интервью риа «новости»', 9.0), ('проведении мероприятий остается', 9.0), ('сильно страдали предприниматели', 9.0), ('самые сложные времена', 9.0), ('вторую мировую войну', 9.0), ('половина европейской части', 9.0), ('занята немецкими войсками', 9.0), ('полчища стояли практически', 9.0), ('мэр москвы добавил', 9.0), ('оборонно-промышленном комплексе', 9.0), ('получении медицинских услуг', 9.0), ('сергей собянин отметил', 9.0), ('любой момент обратиться', 9.0), ('сергей собянин добавил', 9.0), ('работа идет сразу', 9.0), ('оплаты детских садов', 9.0), ('каком формате москва', 9.0), ('которые получит москва', 8.5), ('однако окончательное решение', 8.5), ('бюджет столицы получит', 8.5), ('проведет новогодние мероприятия', 8.5), ('гибли миллионы людей', 8.5), ('сэкономленные средства направить', 8.5), ('вопреки пессимистическим настроениям', 8.333333333333334), ('проведения новогодних мероприятий', 8.0), ('словам мэра москвы', 8

Для лемматизированных текстов все совсем плохо

In [8]:
#corpus_full['rake'] = corpus_full['text_clean'].progress_apply(lambda x: use_rake(x))

In [9]:
def use_TextRank(text):
  kw = keywords.keywords(text, language='russian', additional_stopwords=stop_words, scores=True)
  print(kw)
  list_kw = []
  for k in kw:
    list_kw.append(k[0])

  return ', '.join(list_kw)

#corpus_full['TextRank'] = corpus_full['text'].progress_apply(lambda x: use_TextRank(x))

Здесь получается лучше на лемматизированном тексте

In [10]:
corpus_full['TextRank'] = corpus_full['text_clean'].progress_apply(lambda x: use_TextRank(x))

  0%|          | 0/20 [00:00<?, ?it/s]

[('который получить москва проведение новогодний мероприятие', 0.13812725386800787), ('семья мобилизовать', 0.13205150207101293), ('новый год', 0.12874809269488902), ('поэтому', 0.12154602317527739), ('также', 0.12137755767107558), ('мочь', 0.12070683585608612), ('время', 0.11910009633899624), ('это праздник', 0.11828348353917281), ('помощь', 0.11629805524831176), ('вопреки', 0.1162199139417872), ('столица', 0.11377909055011225), ('поддержка', 0.10782706555362083), ('сергей собянин сказать', 0.10616128826258381), ('важно ребёнок', 0.10358744702846544), ('целый город', 0.10018317981964207), ('сложный', 0.09519458515204174), ('война', 0.09497323043720014), ('получение', 0.0944432731041069), ('детский', 0.09373821416727779), ('человек', 0.09240891174954641), ('вариант', 0.09216758241489104), ('помимо', 0.09090166624922483), ('заявка', 0.08922257350945331), ('важный работа должный', 0.085461415087982), ('гражданин', 0.0837612024121506), ('отказ', 0.08272426622685729), ('решение', 0.0823613

In [11]:
def use_BERT(text):
  kw = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 3), stop_words=stop_words)
  print(kw)
  list_kw = []
  for k in kw:
    list_kw.append(k[0])

  return ', '.join(list_kw)

kw_model = KeyBERT('clips/mfaq')
#corpus_full['BERT'] = corpus_full['text'].progress_apply(lambda x: use_BERT(x))

Downloading:   0%|          | 0.00/1.65k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.74k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/778 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/294 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/464 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:370: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


In [12]:
corpus_full['BERT'] = corpus_full['text_clean'].progress_apply(lambda x: use_BERT(x))

  0%|          | 0/20 [00:00<?, ?it/s]

[('новогодний мероприятие сэкономить', 0.9703), ('который получить москва', 0.9696), ('отметить мэр москва', 0.9695), ('деньга провести новогодний', 0.9681), ('мероприятие сэкономить', 0.968)]
[('здравоохранение', 0.9718), ('фармацевтический', 0.9703), ('контракт сфера медоборудование', 0.9699), ('экономический', 0.9693), ('фармацевтический комплекс стажёр', 0.969)]
[('программа реновация переехать', 0.971), ('реновация переехать', 0.9708), ('переезд новый квартира', 0.9707), ('новый квартира сообщить', 0.9697), ('квартира программа реновация', 0.9695)]
[('год москва появиться', 0.9705), ('новостройка возводить столица', 0.97), ('столица программа реновация', 0.9694), ('удобство житель новостройка', 0.9692), ('москва появиться около', 0.9691)]
[('этап отремонтировать участок', 0.9736), ('первый этап отремонтировать', 0.9731), ('этап капитальный ремонт', 0.973), ('этап отремонтировать', 0.9727), ('провести ремонтновосстановительный', 0.9725)]
[('собянин призвать москвич', 0.9717), ('сер

In [13]:
patterns_POS = []
for tag_set in list(corpus_full['tags_gold']):
  tags = tag_set.split(',')
  for tag in tags:
    tag_ = []
    for word in tag.split(' '):
      if word:
        p = morph.parse(word)[0]
        tag_.append(p.tag.POS)
    patterns_POS.append(' '.join(tag_))
filter_pos = list(set(patterns_POS))
Counter(patterns_POS)

Counter({'ADJF ADJF NOUN': 6,
         'ADJF NOUN': 22,
         'NOUN': 76,
         'NOUN NOUN': 30,
         'NOUN ADJF': 7,
         'NOUN NOUN NOUN': 2,
         'PRTF': 1})

Возьмем одушевленность, так как часть грамемм бессмысленны из-за того, что у нас в двух случаях леммы

In [14]:
patterns_POS_animacy = []
for tag_set in list(corpus_full['tags_gold']):
  tags = tag_set.split(',')
  for tag in tags:
    tag_ = []
    for word in tag.split(' '):
      if word:
        p = morph.parse(word)[0]
        if p.tag.animacy:
          tag_.append(p.tag.POS + ',' + p.tag.animacy)
        else: 
          tag_.append(p.tag.POS)
    patterns_POS_animacy.append(' '.join(tag_))
filter_pos_animacy = list(set(patterns_POS_animacy))
Counter(patterns_POS_animacy)

Counter({'ADJF ADJF NOUN,inan': 6,
         'ADJF NOUN,anim': 2,
         'NOUN,inan': 69,
         'NOUN,anim NOUN,anim': 13,
         'ADJF NOUN,inan': 18,
         'NOUN,anim ADJF': 7,
         'NOUN,anim': 7,
         'NOUN,inan NOUN,inan': 10,
         'ADJF,inan NOUN,inan': 2,
         'NOUN,anim NOUN,inan': 4,
         'NOUN,inan NOUN,inan NOUN,inan': 2,
         'NOUN,inan NOUN,anim': 3,
         'PRTF': 1})

In [15]:
def find_pattern_in_text(text, patterns_pos, patterns_pos_anim):
  d_1_pos, d_2_pos, d_3_pos = {}, {}, {}
  d_1_pos_a, d_2_pos_a, d_3_pos_a = {}, {}, {}

  for token in text.split():
    p = morph.parse(token)[0]
    if p.tag.animacy:
      d_1_pos_a[token] = p.tag.POS + ',' + p.tag.animacy
    else: 
      d_1_pos_a[token] = p.tag.POS
    d_1_pos[token] = p.tag.POS
  
  text_split_2 = [text.split()[i:i+2] for i in range(0, len(text.split()), 2)]
  text_split_3 = [text.split()[i:i+3] for i in range(0, len(text.split()), 3)]

  for tokens in text_split_2:
    tokens_p = []
    tokens_p_a = []
    for token in tokens:
      if token:
        p = morph.parse(token)[0]
        if p.tag.animacy:
          tokens_p_a.append(p.tag.POS + ',' + p.tag.animacy)
        else: 
          tokens_p_a.append(p.tag.POS)
        tokens_p.append(p.tag.POS)

    try:  
      d_2_pos[' '.join(tokens)] = ' '.join(tokens_p)
      d_2_pos_a[' '.join(tokens)] = ' '.join(tokens_p_a)
    except:
      pass

    for tokens in text_split_3:
      tokens_p = []
      tokens_p_a = []
      for token in tokens:
        if token:
          p = morph.parse(token)[0]
          if p.tag.animacy:
            tokens_p_a.append(p.tag.POS + ',' + p.tag.animacy)
          else: 
            tokens_p_a.append(p.tag.POS)
          tokens_p.append(p.tag.POS)

    try:
      d_3_pos[' '.join(tokens)] = ' '.join(tokens_p)
      d_3_pos_a[' '.join(tokens)] = ' '.join(tokens_p_a)
    except:
      pass

    kw = []
    d_1_pos.update(d_2_pos)
    d_1_pos.update(d_3_pos)
    d_1_pos_a.update(d_2_pos_a)
    d_1_pos_a.update(d_3_pos_a)

    for k, v in d_1_pos.items():
      if str(v) in patterns_pos:
        kw.append(k)
    for k, v in d_1_pos_a.items():
      if str(v) in patterns_pos:
        kw.append(k)
    return kw



In [16]:
corpus_full['tags_filter'] = corpus_full['text_clean'].progress_apply(lambda x: find_pattern_in_text(x, patterns_POS, patterns_POS_animacy))

  0%|          | 0/20 [00:00<?, ?it/s]

In [17]:
def filter_kw_POS_anim(tag_set, patterns_pos_anim):
  filtered_tags = []
  tags = tag_set.split(',')
  for tag in tags:
    tag_ = []
    for word in tag.split(' '):
      if word:
        p = morph.parse(word)[0]
        if p.tag.animacy:
          tag_.append(p.tag.POS + ',' + p.tag.animacy)
        else: 
          tag_.append(p.tag.POS)
    try:
      if ' '.join(tag_) in patterns_pos_anim:
        filtered_tags.append(tag)
    except:
      pass

  return ', '.join(filtered_tags)

In [18]:
def filter_kw_POS(tag_set, patterns_pos):
  filtered_tags = []
  tags = tag_set.split(',')
  for tag in tags:
    tag_ = []
    for word in tag.split(' '):
      if word:
        p = morph.parse(word)[0]
        tag_.append(p.tag.POS)
    try:
      if ' '.join(tag_) in patterns_pos:
        filtered_tags.append(tag)
    except:
      pass

  return ', '.join(filtered_tags)

In [19]:
corpus_full['rake_filter_pos'] = corpus_full['rake'].progress_apply(lambda x: filter_kw_POS(x, patterns_POS))
corpus_full['rake_filter_pos_anim'] = corpus_full['rake'].progress_apply(lambda x: filter_kw_POS_anim(x, patterns_POS_animacy))
corpus_full['textrank_filter_pos'] = corpus_full['TextRank'].progress_apply(lambda x: filter_kw_POS(x, patterns_POS))
corpus_full['textrank_filter_pos_anim'] = corpus_full['TextRank'].progress_apply(lambda x: filter_kw_POS_anim(x, patterns_POS_animacy))
corpus_full['bert_filter_pos'] = corpus_full['BERT'].progress_apply(lambda x: filter_kw_POS(x, patterns_POS))
corpus_full['bert_filter_pos_anim'] = corpus_full['BERT'].progress_apply(lambda x: filter_kw_POS_anim(x, patterns_POS_animacy))

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

In [20]:
def evalution_prec(pred, gold, name):
  predicted = pred.split(', ')
  gold_tags = gold.split(', ')

  prec = len(list(set(predicted) & set(gold_tags)))/len(predicted)
  return prec

def evalution_rec(pred, gold, name):
  predicted = pred.split(', ')
  gold_tags = gold.split(', ')

  rec = len(list(set(predicted) & set(gold_tags)))/len(gold_tags)
  return rec

def evalution_f_1(prec, rec, name):
  if prec != 0 or rec != 0:
    f_1 = 2 * prec * rec / (prec + rec)
  else:
    f_1 = 0
  return f_1

In [28]:
corpus_full['tags_gold_clean'] = corpus_full['tags_gold'].progress_apply(lambda x: clear_and_morhp(x, stop_words, morph))

  0%|          | 0/20 [00:00<?, ?it/s]

In [31]:
corpus_full['tags_prec'] = corpus_full.apply(lambda x: evalution_prec(x['tags'], x['tags_gold'], 'tags'), axis=1)
corpus_full['tags_rec'] = corpus_full.apply(lambda x: evalution_rec(x['tags'], x['tags_gold'], 'tags'), axis=1)
corpus_full['tags_f1'] = corpus_full.apply(lambda x: evalution_f_1(x['tags_prec'], x['tags_rec'], 'tags'), axis=1)

corpus_full['tags_manual_prec'] = corpus_full.apply(lambda x: evalution_prec(x['tags_manual'], x['tags_gold'], 'tags_manual'), axis=1)
corpus_full['tags_manual_rec'] = corpus_full.apply(lambda x: evalution_rec(x['tags_manual'], x['tags_gold'], 'tags_manual'), axis=1)
corpus_full['tags_manual_f1'] = corpus_full.apply(lambda x: evalution_f_1(x['tags_manual_prec'], x['tags_manual_rec'], 'tags_manual'), axis=1)

corpus_full['rake_prec'] = corpus_full.apply(lambda x: evalution_prec(x['rake'], x['tags_gold'], 'rake'), axis=1)
corpus_full['rake_rec'] = corpus_full.apply(lambda x: evalution_rec(x['rake'], x['tags_gold'], 'rake'), axis=1)
corpus_full['rake_f1'] = corpus_full.apply(lambda x: evalution_f_1(x['rake_prec'], x['rake_rec'], 'rake'), axis=1)

corpus_full['TextRank_prec'] = corpus_full.apply(lambda x: evalution_prec(x['TextRank'], x['tags_gold'], 'TextRank'), axis=1)
corpus_full['TextRank_rec'] = corpus_full.apply(lambda x: evalution_rec(x['TextRank'], x['tags_gold'], 'TextRank'), axis=1)
corpus_full['TextRank_f1'] = corpus_full.apply(lambda x: evalution_f_1(x['TextRank_prec'], x['TextRank_rec'], 'TextRank'), axis=1)

corpus_full['BERT_prec'] = corpus_full.apply(lambda x: evalution_prec(x['BERT'], x['tags_gold'], 'BERT'), axis=1)
corpus_full['BERT_rec'] = corpus_full.apply(lambda x: evalution_rec(x['BERT'], x['tags_gold'], 'BERT'), axis=1)
corpus_full['BERT_f1'] = corpus_full.apply(lambda x: evalution_f_1(x['BERT_prec'], x['BERT_rec'], 'BERT'), axis=1)

corpus_full['rake_prec_filter_pos'] = corpus_full.apply(lambda x: evalution_prec(x['rake_filter_pos'], x['tags_gold'], 'rake_filter_pos'), axis=1)
corpus_full['rake_rec_filter_pos'] = corpus_full.apply(lambda x: evalution_rec(x['rake_filter_pos'], x['tags_gold'], 'rake_filter_pos'), axis=1)
corpus_full['rake_f1_filter_pos'] = corpus_full.apply(lambda x: evalution_f_1(x['rake_prec_filter_pos'], x['rake_rec_filter_pos'], 'rake_filter_pos'), axis=1)

corpus_full['rake_prec_filter_pos_anim'] = corpus_full.apply(lambda x: evalution_prec(x['rake_filter_pos_anim'], x['tags_gold'], 'rake_filter_pos_anim'), axis=1)
corpus_full['rake_rec_filter_pos_anim'] = corpus_full.apply(lambda x: evalution_rec(x['rake_filter_pos_anim'], x['tags_gold'], 'rake_filter_pos_anim'), axis=1)
corpus_full['rake_f1_filter_pos_anim'] = corpus_full.apply(lambda x: evalution_f_1(x['rake_prec_filter_pos_anim'], x['rake_rec_filter_pos_anim'], 'rake_filter_pos_anim'), axis=1)

corpus_full['textrank_prec_filter_pos'] = corpus_full.apply(lambda x: evalution_prec(x['textrank_filter_pos'], x['tags_gold'], 'TextRank_filter_pos'), axis=1)
corpus_full['textrank_rec_filter_pos'] = corpus_full.apply(lambda x: evalution_rec(x['textrank_filter_pos'], x['tags_gold'], 'TextRank_filter_pos'), axis=1)
corpus_full['textrank_f1_filter_pos'] = corpus_full.apply(lambda x: evalution_f_1(x['textrank_prec_filter_pos'], x['textrank_rec_filter_pos'], 'TextRank_filter_pos'), axis=1)

corpus_full['textrank_prec_filter_pos_anim'] = corpus_full.apply(lambda x: evalution_prec(x['textrank_filter_pos_anim'], x['tags_gold'], 'TextRank_filter_pos_anim'), axis=1)
corpus_full['textrank_rec_filter_pos_anim'] = corpus_full.apply(lambda x: evalution_rec(x['textrank_filter_pos_anim'], x['tags_gold'], 'TextRank_filter_pos_anim'), axis=1)
corpus_full['textrank_f1_filter_pos_anim'] = corpus_full.apply(lambda x: evalution_f_1(x['textrank_prec_filter_pos_anim'], x['textrank_rec_filter_pos_anim'], 'TextRank_filter_pos_anim'), axis=1)

corpus_full['bert_prec_filter_pos'] = corpus_full.apply(lambda x: evalution_prec(x['bert_filter_pos'], x['tags_gold'], 'BERT_filter_pos'), axis=1)
corpus_full['bert_rec_filter_pos'] = corpus_full.apply(lambda x: evalution_rec(x['bert_filter_pos'], x['tags_gold'], 'BERT_filter_pos'), axis=1)
corpus_full['bert_f1_filter_pos'] = corpus_full.apply(lambda x: evalution_f_1(x['bert_prec_filter_pos'], x['bert_rec_filter_pos'], 'BERT_filter_pos'), axis=1)

corpus_full['bert_prec_filter_pos_anim'] = corpus_full.apply(lambda x: evalution_prec(x['bert_filter_pos_anim'], x['tags_gold'], 'BERT_filter_pos_anim'), axis=1)
corpus_full['bert_rec_filter_pos_anim'] = corpus_full.apply(lambda x: evalution_rec(x['bert_filter_pos_anim'], x['tags_gold'], 'BERT_filter_pos_anim'), axis=1)
corpus_full['bert_f1_filter_pos_anim'] = corpus_full.apply(lambda x: evalution_f_1(x['bert_prec_filter_pos_anim'], x['bert_rec_filter_pos_anim'], 'BERT_filter_pos_anim'), axis=1)


In [22]:
corpus_full

,title,text,tags,tags_manual,tags_gold,text_clean,rake,TextRank,BERT,tags_filter,...,textrank_f1_filter_pos,textrank_prec_filter_pos_anim,textrank_rec_filter_pos_anim,textrank_f1_filter_pos_anim,bert_prec_filter_pos,bert_rec_filter_pos,bert_f1_filter_pos,bert_prec_filter_pos_anim,bert_rec_filter_pos_anim,bert_f1_filter_pos_anim
0,Москва направит доходы от новогодних мероприят...,"Доходы, которые получит Москва от проведения н...","голосования, Активный гражданин, Рождество, Се...","Новогодние мероприятия, доходы, поддержка СВО","специальная военная операция, Активный граждан...",доход который получить москва проведение новог...,"будут кратно выше, сергей собянин сказал, инте...",который получить москва проведение новогодний ...,"новогодний мероприятие сэкономить, который пол...","[доход, москва, проведение, мероприятие, эконо...",...,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0
1,Сергей Собянин: Студенты московских вузов и ко...,У студентов московских вузов и колледжей есть ...,"производство, стажировки, фармкомпании, Владис...","Студенты, стажировка, особая экономическая зон...","производство, Владислав Овчинский, cтуденты, с...",студент московский вуз колледж возможность про...,"возможность проходить стажировку, стажеры прин...","компания, москва, оэз, студент, тысяча, столиц...","здравоохранение, фармацевтический, контракт сф...","[студент, вуз, колледж, возможность, стажировк...",...,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0
2,Около 25 тысяч семей воспользовались бесплатно...,В новые квартиры по программе реновации переех...,"новостройки, переезды, программа реновации, Се...","Реновация, переезд, услуги","грузчики, программа реновации, Сергей Левкин, ...",новый квартира программа реновация переехать т...,"севере столицы помощь, помощью суперсервиса «п...","тысяча, услуга, помощь грузчик, переезд, перее...","программа реновация переехать, реновация перее...","[квартира, программа, реновация, тысяча, семья...",...,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0
3,Около 200 тысяч рабочих мест появится в Москве...,К концу 2032 года в Москве появится около 200 ...,"новостройки, рабочие места, нежилые помещения,...","Реновация, рабочие места, вакансии","нежилые помещения, программа реновации, ваканс...",конец год москва появиться около тысяча рабочи...,"удобства жителей новостройки, предприятия сфер...","новый, реализация программа реновация, предпри...","год москва появиться, новостройка возводить ст...","[конец, год, москва, тысяча, рабочий, место, р...",...,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0
4,Начался второй этап капитального ремонта Тульс...,Специалисты комплекса городского хозяйства при...,"эстакады, дороги, капитальный ремонт, ремонт д...","Эстакада, ремонт, демонтаж, эксплуатация","эстакада, демонтаж, Петр Бирюков, дороги, эксп...",специалист комплекс городской хозяйство присту...,"вопросам жилищно-коммунального хозяйства, благ...","хозяйство, этап, работа объект, движение, отре...","этап отремонтировать участок, первый этап отре...","[специалист, комплекс, хозяйство, этап, ремонт...",...,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0
5,Сергей Собянин: Москвичи проголосуют за формат...,"Сергей Собянин предложил москвичам решить, в к...","праздники, Активный гражданин, Рождество, Серг...","Новогодние праздники, голосование, благотварит...","поддержка малого бизнеса , голосование, Активн...",сергей собянин предложить москвич решить какой...,"каком формате москва, проекте «активный гражда...","москва, свой, праздник, человек, город, москви...","собянин призвать москвич, сергей собянин призв...","[сергей, собянин, москвич, формат, москва, год...",...,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0
6,ОЭЗ «Технополис Москва» в 2022 году сдала в ар...,С начала этого года новые предприятия арендова...,"Владимир Ефимов, технополис Москва, аренда пом...","Особая экономическая зона, аренда, производств...","Владислав Овчинский, особая экономическая зона...",начало год новый предприятие арендовать площад...,"будут выпускать препараты, заменители суставно...","москва, оборудование, высокотехнологичный прои

In [32]:
corpus_full.mean()

tags_prec                        0.843333
tags_rec                         0.606071
tags_f1                          0.699620
tags_manual_prec                 0.765000
tags_manual_rec                  0.443730
tags_manual_f1                   0.555759
rake_prec                        0.000000
rake_rec                         0.000000
rake_f1                          0.000000
TextRank_prec                    0.029741
TextRank_rec                     0.079841
TextRank_f1                      0.042073
BERT_prec                        0.000000
BERT_rec                         0.000000
BERT_f1                          0.000000
rake_prec_filter_pos             0.000000
rake_rec_filter_pos              0.000000
rake_f1_filter_pos               0.000000
rake_prec_filter_pos_anim        0.000000
rake_rec_filter_pos_anim         0.000000
rake_f1_filter_pos_anim          0.000000
textrank_prec_filter_pos         0.000000
textrank_rec_filter_pos          0.000000
textrank_f1_filter_pos           0

# Возможные исправления

1. Необходимо как-то урезать длину ключевых слов, так как автоматически выделенные ключевые слова получались в среднем больше 2-х токенов. Проблема в том, что в ручных ключевых словах есть редкие вхождение 3-х токеновых КС, поэтому была необходимость расширить до 3, но это ухудшает качество. 

2. Возможно необходима ручная оценка КС, так как визуально некоторые КС кажутся вполне адекватными, но просто не были включен в золотой стандарт.

3. Оценку нужно усреднять, так как количество выделенных КС зависит от метода.